In [1]:
# conda create --clone pyvizenv --name smallcapcryptosenv -y

In [2]:
# conda activate smallcapcryptosenv

In [3]:
# !conda install -c anaconda seaborn -y

In [4]:
# !pip install pycoingecko

In [4]:
# Initial Imports
import pandas as pd
import os
import requests
import numpy as np
# from dotenv import load_dotenv

from MCForecastTools import MCSimulation
import datetime as dt
import seaborn
import matplotlib.pyplot as plt
import panel as pn
from panel.interact import interact
import plotly.express as px

pn.extension("plotly")
%matplotlib inline

In [6]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [7]:
def time_range(delta=365):
	date_2 = pd.Timestamp("today", tz="America/Chicago") # current date as timestamp
	date_1 = date_2 - pd.to_timedelta(delta, unit="Days")
	
	dates = {
		"date_1": date_1,
		"date_2": date_2}
	return dates
	
dates = time_range()

In [8]:
def unix_date(dates):
	return [dates['date_1'].timestamp(), dates['date_2'].timestamp()]

In [9]:
dates = time_range()
dates

{'date_1': Timestamp('2021-04-02 13:54:55.865392-0500', tz='America/Chicago'),
 'date_2': Timestamp('2022-04-02 13:54:55.865392-0500', tz='America/Chicago')}

In [10]:
from_to = unix_date(dates)
from_to

[1617389695.865392, 1648925695.865392]

In [ ]:
# unix_date(dates)

In [ ]:
# dates = time_range()
# d1_unix = pd.to_datetime(dates["date_1"])#.timestamp()#.map(pd.Timestamp.timestamp)
# print(d1_unix.timestamp())
# dates[date_1.timestamp()]

In [11]:
maple = requests.get('https://api.coingecko.com/api/v3/coins/maple/market_chart/range?vs_currency=usd&from=1617382350.549834&to=1648918350.549834').json()

In [12]:
# maple params list
p_list = {
	'ticker':'maple',
	'request':'market_chart/range',
	'currency':'usd',
	'date_1':1617382350.549834,
	'date_2':1648918350.549834,
}

In [13]:
def make_url(p_list):
	my_url = f"https://api.coingecko.com/api/v3/coins/{p_list['ticker']}/{p_list['request']}?vs_currency={p_list['currency']}&from={p_list['date_1']}&to={p_list['date_2']}"
	return my_url

In [14]:
maple_url = make_url(p_list)
maple_url

'https://api.coingecko.com/api/v3/coins/maple/market_chart/range?vs_currency=usd&from=1617382350.549834&to=1648918350.549834'

In [15]:
maple_json = requests.get(maple_url).json()

In [16]:
maple_df = pd.DataFrame.from_dict(maple_json, orient='columns', dtype=None, columns=None)
maple_df

,prices,market_caps,total_volumes
0,"[1619740800000, 23.327418267811513]","[1619740800000, 0.0]","[1619740800000, 16576656.85415079]"
1,"[1619827200000, 23.327418267811513]","[1619827200000, 0.0]","[1619827200000, 16576656.85415079]"
2,"[1619913600000, 18.915452769785436]","[1619913600000, 0.0]","[1619913600000, 31537632.939870752]"
3,"[1620000000000, 21.309982603593916]","[1620000000000, 0.0]","[1620000000000, 1009195.8446439038]"
4,"[1620086400000, 17.993651917843852]","[1620086400000, 0.0]","[1620086400000, 699889.2299604575]"
...,...,...,...
333,"[1648512000000, 44.45836995899958]","[1648512000000, 205328245.69898155]","[1648512000000, 11288587.509875141]"
334,"[1648598400000, 46.561385905260124]","[1648598400000, 220744734.5592906]","[1648598400000, 7550300.014392077]"
335,"[1648684800000, 52.24895192787693]","[1648684800000, 249319778.6861569]","[1648684800000, 10566379.186666392]"
336,"[1648771200000, 50.970282320420615]","[1648771200000, 242426811.27108622]","[1648771200000, 12544229.03797571]"


In [17]:
# bitcoin params list
p_list = {
	'ticker':'bitcoin',
	'request':'market_chart/range',
	'currency':'usd',
	'date_1':1617382350.549834,
	'date_2':1648918350.549834,
}

In [18]:
bitcoin_url = make_url(p_list)
bitcoin_json = requests.get(bitcoin_url).json()
bitcoin_df = pd.DataFrame.from_dict(bitcoin_json, orient='columns', dtype=None, columns=None)

In [19]:
bitcoin_df

,prices,market_caps,total_volumes
0,"[1617408000000, 59060.02678333801]","[1617408000000, 1103133931136.5508]","[1617408000000, 52224918752.08043]"
1,"[1617494400000, 57060.40665471088]","[1617494400000, 1067101827372.9529]","[1617494400000, 52067853987.600494]"
2,"[1617580800000, 58228.802623466196]","[1617580800000, 1087344280109.6896]","[1617580800000, 46156767978.177345]"
3,"[1617667200000, 58706.81559311396]","[1617667200000, 1096202611389.9541]","[1617667200000, 54670173277.08731]"
4,"[1617753600000, 58062.17872433649]","[1617753600000, 1084341670320.8142]","[1617753600000, 61497704158.13829]"
...,...,...,...
360,"[1648512000000, 46994.879118357465]","[1648512000000, 892747620147.7012]","[1648512000000, 30051327357.066643]"
361,"[1648598400000, 47459.261237539096]","[1648598400000, 901430001888.9999]","[1648598400000, 26128040680.825665]"
362,"[1648684800000, 47063.36584996355]","[1648684800000, 895356010540.8462]","[1648684800000, 24497248306.346912]"
363,"[1648771200000, 45528.40715313835]","[1648771200000, 865010142444.9791]","[1648771200000, 27736469645.537262]"


In [5]:
mantra_dao = requests.get('https://api.coingecko.com/api/v3/coins/mantra-dao/market_chart/range?vs_currency=usd&from=1617382350.549834&to=1648918350.549834').json()
# mantra_dao params list
p_list = {
	'ticker':'mantra-dao',
	'request':'market_chart/range',
	'currency':'usd',
	'date_1':1617382350.549834,
	'date_2':1648918350.549834,
}
def make_url(p_list):
	my_url = f"https://api.coingecko.com/api/v3/coins/{p_list['ticker']}/{p_list['request']}?vs_currency={p_list['currency']}&from={p_list['date_1']}&to={p_list['date_2']}"
	return my_url
mantra_dao_url = make_url(p_list)
mantra_dao_url
mantra_dao_json = requests.get(mantra_dao_url).json()
mantra_dao_df = pd.DataFrame.from_dict(mantra_dao_json, orient='columns', dtype=None, columns=None)
mantra_dao_df

,prices,market_caps,total_volumes
0,"[1617408000000, 0.49180304251853346]","[1617408000000, 139768113.18269786]","[1617408000000, 20549413.50142256]"
1,"[1617494400000, 0.44703741571754596]","[1617494400000, 127646464.35498866]","[1617494400000, 23327360.94054037]"
2,"[1617580800000, 0.4705822593249223]","[1617580800000, 133290620.81353465]","[1617580800000, 18511958.391771797]"
3,"[1617667200000, 0.44774137889491183]","[1617667200000, 128714814.48541008]","[1617667200000, 18246513.66924431]"
4,"[1617753600000, 0.4546352254274566]","[1617753600000, 129559441.27914387]","[1617753600000, 20105456.082223833]"
...,...,...,...
360,"[1648512000000, 0.08544219784014513]","[1648512000000, 38537315.135362506]","[1648512000000, 14111550.759559585]"
361,"[1648598400000, 0.08684764601900058]","[1648598400000, 38934037.33925703]","[1648598400000, 4376712.013833306]"
362,"[1648684800000, 0.08802154191964097]","[1648684800000, 39390571.67910122]","[1648684800000, 2553283.7613220857]"
363,"[1648771200000, 0.08711279909768288]","[1648771200000, 38967843.97296452]","[1648771200000, 2981136.0426940345]"


In [6]:
gnosis = requests.get('https://api.coingecko.com/api/v3/coins/gnosis/market_chart/range?vs_currency=usd&from=1617382350.549834&to=1648918350.549834').json()
# gnosis params list
p_list = {
	'ticker':'gnosis',
	'request':'market_chart/range',
	'currency':'usd',
	'date_1':1617382350.549834,
	'date_2':1648918350.549834,
}
def make_url(p_list):
	my_url = f"https://api.coingecko.com/api/v3/coins/{p_list['ticker']}/{p_list['request']}?vs_currency={p_list['currency']}&from={p_list['date_1']}&to={p_list['date_2']}"
	return my_url
gnosis_url = make_url(p_list)
gnosis_url
gnosis_json = requests.get(gnosis_url).json()
gnosis_df = pd.DataFrame.from_dict(gnosis_json, orient='columns', dtype=None, columns=None)
gnosis_df

,prices,market_caps,total_volumes
0,"[1617408000000, 171.1864882344057]","[1617408000000, 257412259.82662484]","[1617408000000, 1649443.6807593077]"
1,"[1617494400000, 167.61902387959773]","[1617494400000, 252159546.41656566]","[1617494400000, 1843098.231934098]"
2,"[1617580800000, 170.1368539050166]","[1617580800000, 256009842.02616218]","[1617580800000, 2034565.490268035]"
3,"[1617667200000, 172.42619490922652]","[1617667200000, 259356663.1181328]","[1617667200000, 1527717.4479498384]"
4,"[1617753600000, 172.06218992666948]","[1617753600000, 258986206.29684398]","[1617753600000, 1913209.8714035456]"
...,...,...,...
360,"[1648512000000, 456.8787737969117]","[1648512000000, 852091249.4354886]","[1648512000000, 121819106.45700967]"
361,"[1648598400000, 477.1236539569757]","[1648598400000, 889182902.2649273]","[1648598400000, 74059671.37212028]"
362,"[1648684800000, 465.6238001873581]","[1648684800000, 869221723.3279495]","[1648684800000, 19128685.630640037]"
363,"[1648771200000, 422.35733691389066]","[1648771200000, 787707837.526234]","[1648771200000, 16793739.703159537]"


In [7]:
dash = requests.get('https://api.coingecko.com/api/v3/coins/DASH/market_chart/range?vs_currency=usd&from=1617382350.549834&to=1648918350.549834').json()
# dash params list
p_list = {
	'ticker':'dash',
	'request':'market_chart/range',
	'currency':'usd',
	'date_1':1617382350.549834,
	'date_2':1648918350.549834,
}
def make_url(p_list):
	my_url = f"https://api.coingecko.com/api/v3/coins/{p_list['ticker']}/{p_list['request']}?vs_currency={p_list['currency']}&from={p_list['date_1']}&to={p_list['date_2']}"
	return my_url
dash_url = make_url(p_list)
dash_url
dash_json = requests.get(dash_url).json()
dash_df = pd.DataFrame.from_dict(dash_json, orient='columns', dtype=None, columns=None)
dash_df

,prices,market_caps,total_volumes
0,"[1617408000000, 243.8888680553035]","[1617408000000, 2459321754.778395]","[1617408000000, 1295297602.4227169]"
1,"[1617494400000, 223.04032525753365]","[1617494400000, 2261937297.5270853]","[1617494400000, 963632684.5646185]"
2,"[1617580800000, 271.0111995137088]","[1617580800000, 2731399483.168387]","[1617580800000, 1973664233.2614198]"
3,"[1617667200000, 273.7343457085849]","[1617667200000, 2754876057.7440042]","[1617667200000, 1619966678.6661909]"
4,"[1617753600000, 277.6925581550838]","[1617753600000, 2795151490.0978127]","[1617753600000, 1609952338.4646494]"
...,...,...,...
360,"[1648512000000, 127.40939445204293]","[1648512000000, 1359848202.4739077]","[1648512000000, 342117558.73248506]"
361,"[1648598400000, 130.4179099926202]","[1648598400000, 1389191923.0114956]","[1648598400000, 355960651.4849238]"
362,"[1648684800000, 132.68611553761065]","[1648684800000, 1417941088.9726298]","[1648684800000, 305017779.4584622]"
363,"[1648771200000, 127.11884751896619]","[1648771200000, 1354424346.0254216]","[1648771200000, 366866483.87184143]"


In [9]:
knc = requests.get('https://api.coingecko.com/api/v3/coins/kyber-network-crystal/market_chart/range?vs_currency=usd&from=1617382350.549834&to=1648918350.549834').json()
# knc params list
p_list = {
	'ticker':'kyber-network-crystal',
	'request':'market_chart/range',
	'currency':'usd',
	'date_1':1617382350.549834,
	'date_2':1648918350.549834,
}
def make_url(p_list):
	my_url = f"https://api.coingecko.com/api/v3/coins/{p_list['ticker']}/{p_list['request']}?vs_currency={p_list['currency']}&from={p_list['date_1']}&to={p_list['date_2']}"
	return my_url
knc_url = make_url(p_list)
knc_url
knc_json = requests.get(knc_url).json()
knc_df = pd.DataFrame.from_dict(knc_json, orient='columns', dtype=None, columns=None)
knc_df

,prices,market_caps,total_volumes
0,"[1619049600000, 2.92880376121845]","[1619049600000, 0.0]","[1619049600000, 9426.254133309052]"
1,"[1619136000000, 2.92880376121845]","[1619136000000, 0.0]","[1619136000000, 9426.254133309052]"
2,"[1621382400000, 1.9156537042767416]","[1621382400000, 0.0]","[1621382400000, 369565.93852236617]"
3,"[1621468800000, 1.9156537042767416]","[1621468800000, 0.0]","[1621468800000, 369565.93852236617]"
4,"[1621555200000, 2.1150940633852344]","[1621555200000, 0.0]","[1621555200000, 272393.4743117607]"
...,...,...,...
316,"[1648512000000, 3.472223575667224]","[1648512000000, 357234659.44742733]","[1648512000000, 32186530.13688679]"
317,"[1648598400000, 3.4562141740656234]","[1648598400000, 354341665.72390515]","[1648598400000, 58543230.954627596]"
318,"[1648684800000, 3.439994972520075]","[1648684800000, 352677239.72539794]","[1648684800000, 23866693.63664669]"
319,"[1648771200000, 3.2513767508434617]","[1648771200000, 332798814.04062605]","[1648771200000, 33506526.878218416]"


In [10]:
mco2 = requests.get('https://api.coingecko.com/api/v3/coins/moss-carbon-credit/market_chart/range?vs_currency=usd&from=1617382350.549834&to=1648918350.549834').json()
# mco2 params list
p_list = {
	'ticker':'moss-carbon-credit',
	'request':'market_chart/range',
	'currency':'usd',
	'date_1':1617382350.549834,
	'date_2':1648918350.549834,
}
def make_url(p_list):
	my_url = f"https://api.coingecko.com/api/v3/coins/{p_list['ticker']}/{p_list['request']}?vs_currency={p_list['currency']}&from={p_list['date_1']}&to={p_list['date_2']}"
	return my_url
mco2_url = make_url(p_list)
mco2_url
mco2_json = requests.get(mco2_url).json()
mco2_df = pd.DataFrame.from_dict(mco2_json, orient='columns', dtype=None, columns=None)
mco2_df

,prices,market_caps,total_volumes
0,"[1617408000000, 16.511271412578765]","[1617408000000, 0.0]","[1617408000000, 660481.7082343155]"
1,"[1617494400000, 16.524784497245562]","[1617494400000, 0.0]","[1617494400000, 755013.3653798351]"
2,"[1617580800000, 16.302679311504967]","[1617580800000, 0.0]","[1617580800000, 281572.9596885771]"
3,"[1617667200000, 16.254631449350978]","[1617667200000, 0.0]","[1617667200000, 468543.8888360203]"
4,"[1617753600000, 16.61997434727368]","[1617753600000, 0.0]","[1617753600000, 215059.40373830596]"
...,...,...,...
360,"[1648512000000, 12.134499271690677]","[1648512000000, 0.0]","[1648512000000, 3306325.8061994705]"
361,"[1648598400000, 10.478961075403147]","[1648598400000, 0.0]","[1648598400000, 2410580.090265389]"
362,"[1648684800000, 10.489785321253303]","[1648684800000, 0.0]","[1648684800000, 579106.267422481]"
363,"[1648771200000, 10.447934305200054]","[1648771200000, 0.0]","[1648771200000, 558835.0846713678]"


In [11]:
orca = requests.get('https://api.coingecko.com/api/v3/coins/orca/market_chart/range?vs_currency=usd&from=1617382350.549834&to=1648918350.549834').json()
# orca params list
p_list = {
	'ticker':'orca',
	'request':'market_chart/range',
	'currency':'usd',
	'date_1':1617382350.549834,
	'date_2':1648918350.549834,
}
def make_url(p_list):
	my_url = f"https://api.coingecko.com/api/v3/coins/{p_list['ticker']}/{p_list['request']}?vs_currency={p_list['currency']}&from={p_list['date_1']}&to={p_list['date_2']}"
	return my_url
orca_url = make_url(p_list)
orca_url
orca_json = requests.get(orca_url).json()
orca_df = pd.DataFrame.from_dict(orca_json, orient='columns', dtype=None, columns=None)
orca_df

,prices,market_caps,total_volumes
0,"[1628553600000, 3.2906839918038306]","[1628553600000, 17352251.441349793]","[1628553600000, 603998.4653276094]"
1,"[1628640000000, 3.2906839918038306]","[1628640000000, 17352251.441349793]","[1628640000000, 603998.4653276094]"
2,"[1628726400000, 2.7114769557376417]","[1628726400000, 15671879.592141949]","[1628726400000, 256044.76893030552]"
3,"[1628812800000, 2.7026734761646978]","[1628812800000, 14173838.441674411]","[1628812800000, 120752.74824156254]"
4,"[1628899200000, 2.6706407437193893]","[1628899200000, 14028469.279429479]","[1628899200000, 1114745.013565733]"
...,...,...,...
231,"[1648512000000, 2.312397618171142]","[1648512000000, 39717124.9866681]","[1648512000000, 3361438.8485772703]"
232,"[1648598400000, 2.3120212572269243]","[1648598400000, 39456597.134906024]","[1648598400000, 11101122.931270245]"
233,"[1648684800000, 2.370241913857492]","[1648684800000, 40663185.31786407]","[1648684800000, 7224704.038772703]"
234,"[1648771200000, 2.3279968467894188]","[1648771200000, 39722329.53973195]","[1648771200000, 6352373.675971462]"


In [12]:
wcfg = requests.get('https://api.coingecko.com/api/v3/coins/wrapped-centrifuge/market_chart/range?vs_currency=usd&from=1617382350.549834&to=1648918350.549834').json()
# wcfg params list
p_list = {
	'ticker':'wrapped-centrifuge',
	'request':'market_chart/range',
	'currency':'usd',
	'date_1':1617382350.549834,
	'date_2':1648918350.549834,
}
def make_url(p_list):
	my_url = f"https://api.coingecko.com/api/v3/coins/{p_list['ticker']}/{p_list['request']}?vs_currency={p_list['currency']}&from={p_list['date_1']}&to={p_list['date_2']}"
	return my_url
wcfg_url = make_url(p_list)
wcfg_url
wcfg_json = requests.get(wcfg_url).json()
wcfg_df = pd.DataFrame.from_dict(wcfg_json, orient='columns', dtype=None, columns=None)
wcfg_df

,prices,market_caps,total_volumes
0,"[1626220800000, 0.7290555572563795]","[1626220800000, 0.0]","[1626220800000, 3192553.0990898516]"
1,"[1626307200000, 0.7290555572563795]","[1626307200000, 0.0]","[1626307200000, 3192553.0990898516]"
2,"[1626393600000, 0.7058330673903913]","[1626393600000, 0.0]","[1626393600000, 4856349.791530815]"
3,"[1626480000000, 0.8054724000680012]","[1626480000000, 0.0]","[1626480000000, 4919025.038992263]"
4,"[1626566400000, 0.8503875093442439]","[1626566400000, 0.0]","[1626566400000, 3612821.4268891006]"
...,...,...,...
258,"[1648512000000, 0.5101842635358531]","[1648512000000, 102030517.6782886]","[1648512000000, 2262528.4926103507]"
259,"[1648598400000, 0.5149996673479013]","[1648598400000, 102856487.2124544]","[1648598400000, 2981703.147989845]"
260,"[1648684800000, 0.549268185568001]","[1648684800000, 105609234.42520435]","[1648684800000, 2177282.504634925]"
261,"[1648771200000, 0.5144533379146247]","[1648771200000, 103367870.09507123]","[1648771200000, 2726204.944048285]"


In [13]:
wluna = requests.get('https://api.coingecko.com/api/v3/coins/luna-wormhole/market_chart/range?vs_currency=usd&from=1617382350.549834&to=1648918350.549834').json()
# wluna params list
p_list = {
	'ticker':'luna-wormhole',
	'request':'market_chart/range',
	'currency':'usd',
	'date_1':1617382350.549834,
	'date_2':1648918350.549834,
}
def make_url(p_list):
	my_url = f"https://api.coingecko.com/api/v3/coins/{p_list['ticker']}/{p_list['request']}?vs_currency={p_list['currency']}&from={p_list['date_1']}&to={p_list['date_2']}"
	return my_url
wluna_url = make_url(p_list)
wluna_url
wluna_json = requests.get(wluna_url).json()
wluna_df = pd.DataFrame.from_dict(wluna_json, orient='columns', dtype=None, columns=None)
wluna_df

,prices,market_caps,total_volumes
0,"[1646006400000, 89.30088502934669]","[1646006400000, 0.0]","[1646006400000, 290993.561484965]"
1,"[1646092800000, 89.30088502934669]","[1646092800000, 0.0]","[1646092800000, 290993.561484965]"
2,"[1646179200000, 91.4220242088813]","[1646179200000, 0.0]","[1646179200000, 391740.880756261]"
3,"[1646265600000, 91.21671530102458]","[1646265600000, 0.0]","[1646265600000, 195024.7915909496]"
4,"[1646352000000, 90.9647530942314]","[1646352000000, 0.0]","[1646352000000, 194768.86736261687]"
5,"[1646438400000, 86.06110327688765]","[1646438400000, 0.0]","[1646438400000, 351899.87941278913]"
6,"[1646524800000, 87.63916850282531]","[1646524800000, 0.0]","[1646524800000, 178787.58281127768]"
7,"[1646611200000, 79.1157004276018]","[1646611200000, 0.0]","[1646611200000, 10103.199266565407]"
8,"[1646697600000, 77.2219624873325]","[1646697600000, 0.0]","[1646697600000, 204671.91816689176]"
9,"[1646784000000, 85.24606121461393]","[1646784000000, 0.0]","[1646784000000, 150741.70378662244]"
